# Big Data Processing
## *1) броене*

Имаш файл, в който са записани 1 милиард 32-битови числа (като
uint32_t, едно след друго).

а) да се преброи колко са уникалните числа. Пример:

    1 1 2 2 -> 2
    1 1 1 1 -> 1
    1 2 3 3 -> 3

б) да се преброи колко числа се срещат само веднъж.


In [ ]:
import random, operator

maxN = 10001 # 2**32-1
Nlim = 1001
numslist = []

print('...run')

for _ in range(1,Nlim):
    numslist.append(str(random.randint(1, maxN)))

nlist = {}
for num in numslist:
    if num not in nlist:
        nlist[num] = 0
    nlist[num] += 1

nlist_sorted = sorted(nlist.items(), key=lambda kv: kv[1], reverse=True)
#print(nlist_sorted)

unique_nums = len(nlist)
onlyOne_nums = 0
for key in nlist: 
    if nlist[key] == 1: 
        onlyOne_nums += 1
        
print('From Unique nums = %d  Only-One nums = %d' % (unique_nums, onlyOne_nums))
print('...end')

Колкото е по-голяма maxN, толкова по-малко е повторението на дадено случайно число.

При максималното $2**32-1$ практически нямаме повторения на случайните числа!

## *2) fizzbuzz*

Стандартната задача fizzbuzz, с уловка:

Да се напише програма, която извежда числата от 1 до 100. Ако числото
се дели на 3, вместо него да печата A, ако се дели на 5, вместо него да печата B, ако се дели на 15 - AB. Да се напише без никакви conditional-и/проверки, вкл. без for/while loop конструкции.

    for num in range(1,21):
        string = ""
        if num % 3 == 0:
            string = string + "A"
        if num % 5 == 0:
            string = string + "B"
        if num % 5 != 0 and num % 3 != 0:
            string = string + str(num)
        print(string)

In [ ]:
fizzbuzz = list(map(lambda i: "A"*(not i%3)+"B"*(not i%5)+str(i)*(i%3!=0 and i%5!=0),[*range(1,101)]))
print(fizzbuzz)

## *3) анализ*

На [bigf.json.bz2](https://.../bigf.json.bz2) (big json file 25 GB, без ) има един компресиран json файл. В него има описани дискове с модел и сериен номер. Да се преброи колко са моделите и колко пъти се среща всеки от тях.

In [ ]:
import json

def find_all_newlines(input_str, search_str):
    l1 = [0]
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1


if __name__ == "__main__":
    print('...run')

    fin = open("./bigf.json", "rt", buffering=1500)
    dskList = []
    snmList = []
    nxbuf = []
    count = 0
    prev_id = -1   # id start from 0

    while True:
        buf = fin.read(1024)
        if buf:
            if nxbuf != []:
                buf = str(nxbuf) + buf
                buf = buf.lstrip()

            rpbuf = buf.replace('[', '')
            buf  = rpbuf.replace(']', '')
            rpbuf = buf.replace('},', '}\n')
            
            nidx = find_all_newlines(rpbuf, '\n')

            jsonBuf = rpbuf[:nidx[-1]]
            
            for i in range(0,len(nidx)-1):
                st = jsonBuf[nidx[i]:nidx[i+1]]
                row = json.loads(st)
                dskList.append(row['model'])
                snmList.append(row['serial'])

                new_id = row['id']    # sequence test
                if new_id == prev_id+1:
                    prev_id = new_id
                else:
                    print('\n!!! id Error ', new_id)
                    break
            
            count += len(nidx)
            if count > 1000000: break
            
            nxbuf = rpbuf[nidx[-1]:]     # last part of buffer

        else:
            break

    fin.close()

    res = {} 
    for key in snmList: 
        for value in dskList: 
            res[key] = value 
            break

    ds = {}
    for model in dskList:
        if model not in ds:
            ds[model] = 0
        ds[model] += 1
    print(ds)
    model_freq = []
    for key, value in ds.items():
        model_freq.append((value, key))
    model_freq.sort(reverse=True)

    print('   count records = ', count)
    print('       all disks = ', len(dskList))
    print('uniq disk models = ', len(model_freq))

    # print(model_freq)

    print('...ready')